In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [4]:
base=pd.read_csv('PETR4.SA.csv')
base=base.dropna()
base_treinamento=base.iloc[:, 1:2].values

In [8]:
normalizador=MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [9]:
previsores=[]
preco_real=[]
for i in range(90,850):
    previsores.append(base_treinamento_normalizada[i-90:i,0])
    preco_real.append(base_treinamento_normalizada[i,0])

In [10]:
previsores,preco_real=np.array(previsores), np.array(preco_real)

In [13]:
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))

In [17]:
regressor=Sequential()
regressor.add(LSTM(units=100,return_sequences=True, input_shape= (previsores.shape[1],1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1,activation='linear'))

regressor.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=['mean_absolute_error'])

regressor.fit(previsores,preco_real, epochs=100, batch_size=32)

Epoch 1/100


c:\Users\samue\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 109ms/step - loss: 0.0880 - mean_absolute_error: 0.2246
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - loss: 0.0222 - mean_absolute_error: 0.1136
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - loss: 0.0235 - mean_absolute_error: 0.1204
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - loss: 0.0158 - mean_absolute_error: 0.0973
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - loss: 0.0145 - mean_absolute_error: 0.0939
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 105ms/step - loss: 0.0187 - mean_absolute_error: 0.1082
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - loss: 0.0134 - mean_absolute_error: 0.0888
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - loss: 0.0172 - mean_absolute_error: 0.1032
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - loss: 0.0122 - mean_absolute_error: 0.0843
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - loss: 0.0126 - mean_absolute_error: 0.0876
Epoch 11/100
24/24 ━━━━━━━━━━━━

In [27]:
base_teste=pd.read_csv('PETR4.SAJan.csv')
preco_real_teste=base_teste.iloc[:,1:2].values
base_completa=pd.concat((base['Open'], base_teste['Open']), axis=0)
entradas=base_completa[len(base_completa)-len(base_teste)-90:].values
entradas=entradas.reshape(-1,1)
entradas=normalizador.transform(entradas)

In [28]:
X_teste=[]
for i in range(90,112):
    X_teste.append(entradas[i-90:i,0])
X_teste=np.array(X_teste)
X_teste=np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1],1))

In [29]:
previsoes=regressor.predict(X_teste)
previsoes=normalizador.inverse_transform(previsoes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
